In [6]:
import sys
sys.path.append("../")
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
import numpy as np
import random
import time 
from core.data_handler import *
from core.inference_handler import *
%load_ext autoreload
%autoreload 2

# Simple code to generate a new map
def cleanMap(center, boundary): 
    # Mark transformer
    tMarker =  Marker(location=center)
    # Mark polygon
    tPoly = Polygon(locations=boundary, weight=2,
                color='#003d99', opacity=0.4, fill_opacity=0.0,
                fill_color='#ccffcc') 
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=center ,zoom=16)
    #tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

## Connect to db 

In [7]:
conn = connect_to_psql(dbname='gis', user='jennyzhou')

Connection established. DNS info: dbname=gis user=jennyzhou


## Define the Freimann subarea

In [10]:
# Define freimann subarea 
area_id = 3
core_center, core_boundary = select_freimann_subarea(area_id=area_id, conn=conn)
# tMap = cleanMap(core_center, core_boundary)
# tMap

Select freimann subarea with id=3


## Generate groundtruth step by step

### Step0. Merge duplicated transformers and cabinets and store into database( Skip this if they are already in DB)

In [12]:
# Select transformers, cabinets
transformers = select_transformers(area_id, conn)
cabinets = select_cabinets(area_id, conn)
print 'Transformers: %d \nCabinets: %d \n'% (len(transformers), len(cabinets))

# Merge duplicate transformers and cabinets i.e. within distance threshold
origin_trans = list(transformers)
merged_trans = list(merge_nodes(origin_trans, threshold=1e-4).keys())
print 'Original transformers %d  Merged transformers %d'%(len(transformers), len(merged_trans))

origin_cabs = list(cabinets)
merged_cabs = list(merge_nodes(origin_cabs, threshold=5e-3).keys())
print 'Original cabinets %d  Merged cabinets %d'%(len(cabinets), len(merged_cabs))

# Save merged transformers and cabinets in table freimann_processed_nodes
# insert_merged_nodes(area_id, merged_trans, merged_cabs, conn)  # Uncomment this line for really saving the result

Transformers: 4 
Cabinets: 15 

Merging time 0.000434160232544 s
Original transformers 4  Merged transformers 1
Merging time 0.0110239982605 s
Original cabinets 15  Merged cabinets 14


### Step1.  Join transformers and cabinets with cables endpoints

In [13]:
# Select merged transformers, cabinets
merged_trans, merged_cabs = select_merged_trans_cabs(area_id, conn)
print 'Transformers: %d \nCabinets: %d'% (len(merged_trans), len(merged_cabs))

# Select NSP Cables
cables = select_cables(area_id, conn)
print 'Cables: %d'%len(cables)

# Join transformers and cabinets with cables endpoints
origin_cables = list(cables)
target_nodes = list(merged_cabs + merged_trans)
joint_cables = join_cables(origin_cables, target_nodes, threshold=5e-3)

# Uncomment the following lines to visualize the merged nodes and joint cables
# tMap = cleanMap(core_center, core_boundary)
# for tp in merged_trans:  
#     cs = Circle(location=tp, weight=2, opacity = 0.7, color = 'red', radius = 1) 
#     tMap.add_layer(cs)
    
# for tp in merged_cabs:
#     cs = Circle(location=tp, weight=2, opacity = 0.7, color = 'green', radius = 1) 
#     tMap.add_layer(cs)

# for cable in joint_cables:
#     pl = Polyline(locations=cable, weight=1, color='blue', opacity=0.7)
#     tMap.add_layer(pl)

# tMap

Transformers: 1 
Cabinets: 14
Cables: 243
Target nodes: 15
Joining cables took: 0.392870903015 s; Joint endpoints: 95


### Step2. Generate graph

In [18]:
# Generate connected graphs with joint edges
subgraphs = generate_connected_subgraph(joint_cables)
print 'Generated subgraphs:',len(subgraphs)

# Select the graph with most edges
lens = [len(graph.edges()) for graph in subgraphs]
graph = subgraphs[np.argsort(lens)[-1]]
print 'Largest Graph edges: %d nodes: %d connected: %s'%(len(graph.edges()), len(graph.nodes()), nx.is_connected(graph))   

# Uncomment the following lines to visualize each subgraph with different color
# tMap = cleanMap(core_center, core_boundary)
# for graph in subgraphs:
#     color = "#%06x" % random.randint(0, 0xFFFFFF)
#     for tp in graph.nodes():
#         cs = Circle(location=tp, weight=1, opacity = 0.9, color = 'red', radius = 0) 
#         tMap.add_layer(cs)

#     for line in graph.edges():
#         pl = Polyline(locations=line, weight=2, color=color, opacity=0.7)
#         tMap.add_layer(pl)
# tMap

Generated subgraphs: 12
Largest Graph edges: 1253 nodes: 1238 connected: True


### Step3. Simplify the graph and save graph result

In [20]:
# Reduce duplicated nodes and edges
reduced_graph = reduce_graph(graph)

# Save graph
sav_graph_npy('../graph/aid_%d_gt.npy'%area_id, reduced_graph)

# Uncomment the following lines to visualize final reduced graph
# tMap = cleanMap(core_center, core_boundary)
# for tp in reduced_graph.nodes():
#     cs = Circle(location=tp, weight=1, opacity = 0.9, color = 'red', radius = 0) 
#     tMap.add_layer(cs)

# for line in reduced_graph.edges():
#     pl = Polyline(locations=line, weight=1, color='blue', opacity=0.7)
#     tMap.add_layer(pl)
# tMap

Reducing graph took: 31.9054160118 Reduced Graph: 758 edges, 725 nodes Connected: True
Save graph as ../graph/aid_3_gt.npy


## Load saved graph and save as csv for visualization in QGIS

In [5]:
G, total_length = load_graph_npy('../graph/aid_%d_gt.npy'%area_id)
print 'Graph edge length: %s'%total_length
# Save G as csv file
sav_graph_to_csv(save_dir='../graph', prefix='aid_%d_gt'%area_id, graph=G) 

Graph edge length: 3886.88244649
Save to ../graph/aid_3_gt_nodes.csv
Save to ../graph/aid_3_gt_edges.csv
